In [1]:
import torch
from tqdm import tqdm
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import ImageFolder
import numpy as np
import os
import torchvision.transforms as transform
from torchvision.models import resnet18

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
transform_train = transforms.Compose([
    # transform.randomCrop()
    transforms.Resize((299,299)),
    transform.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transform.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225]),
    # Cutout(n_holes=1,length=16),
])

In [3]:
transform_val = transforms.Compose([
    transforms.Resize((299,299)),
    transforms.ToTensor(),
    transform.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

In [4]:
transform_test = transforms.Compose([
    transforms.Resize((299,299)),
    transforms.ToTensor(),
    transform.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [6]:
batch_size = 32

In [13]:
output_path = "/content/drive/MyDrive/Datasets/Ham10000/ham10000_reduction_spilt"
train_dataset = ImageFolder(os.path.join(output_path, "train"), transform= transform_train)
val_dataset = ImageFolder(os.path.join(output_path, "val"), transform= transform_val)
test_dataset = ImageFolder(os.path.join(output_path, "test"), transform= transform_test)
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
val_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle = True)

# Load Model

In [14]:
n_class = 7
model = resnet18()

model.conv1 = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size= 7, stride= 1, padding= 1, bias=False)
model.fc = torch.nn.Linear(512, n_class)
model = model.to(device)
criterion = nn.CrossEntropyLoss().to(device)

# Train and Validate Model

In [15]:
n_epochs = 250
valid_loss_min = np.Inf
accuracy = []
lr = 0.1
counter = 0

In [ ]:
for epoch in tqdm(range(1, n_epochs+1)):

    # Initilise loss value
    train_loss = 0.0
    valid_loss = 0.0
    total_sample = 0
    right_sample = 0

    #Dynamically changing learning rates
    if epoch < 140:
        lr = 0.1
    elif epoch < 200:
        lr = 0.01
    elif epoch < 250:
        lr = 0.001
    else:
        lr = 0.0001

    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    model.train()
    for data, target in train_dataloader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() #clear gradient
        output= model(data).to(device)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    model.eval()
    for data, target in val_dataloader:
      data, target = data.to(device), target.to(device)
      output= model(data).to(device)
      loss = criterion(output, target)
      valid_loss += loss.item()*data.size(0)
      _, pred = torch.max(output, 1)
      correct_tensor = pred.eq(target.data.view_as(pred))
      total_sample += batch_size
      for i in correct_tensor:
        if i:
          right_sample += 1
    print("Accuracy:", 100*right_sample/total_sample, "%")
    accuracy.append(right_sample/total_sample)

    train_loss = train_loss/len(train_dataloader.dataset)
    valid_loss = valid_loss/len(val_dataloader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))

    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/2024/COMP6016 - Computer Science Project 2/checkpoint/resnet18_skincancer.pt')
        valid_loss_min = valid_loss
        counter = 0
    else:
        counter += 1

  0%|          | 1/250 [58:14<241:41:02, 3494.23s/it]

Accuracy: 65.19886363636364 %
Epoch: 1 	Training Loss: 1.371972 	Validation Loss: 1.056952
Validation loss decreased (inf --> 1.056952).  Saving model ...


  1%|          | 2/250 [1:41:46<204:58:11, 2975.37s/it]

Accuracy: 58.38068181818182 %
Epoch: 2 	Training Loss: 1.004729 	Validation Loss: 1.118417


  1%|          | 3/250 [2:25:08<192:27:21, 2805.03s/it]

Accuracy: 57.95454545454545 %
Epoch: 3 	Training Loss: 0.964514 	Validation Loss: 1.101732


# Test model

In [ ]:
model = resnet18()
model.conv1 = nn.conv2d(in_channels = 7, out_channels = 64, kernel_size= 7, stride= 1, padding= 1, bias=False)
model.fc = torch.nn.Linear(512, n_class)

model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/2024/COMP6016 - Computer Science Project 2/checkpoint/resnet18_skincancer.pt'))
model = model.to(device)

total_sample = 0
right_sample = 0

for data, target in test_dataloader:
  data, target = data.to(device), target.to(device)
  output= model(data).to(device)
  _, pred = torch.max(output, 1)
  correct_tensor = pred.eq(target.data.view_as(pred))
  total_sample += batch_size
  for i in correct_tensor:
    if i:
      right_sample += 1

print("Accuracy:", 100*right_sample/total_sample, "%")